In [1]:
import numpy as np
import pandas as pd
from datasetsforecast.m4 import M4, M4Info
from datasetsforecast.m3 import M3, M3Info
from coreforecast.scalers import boxcox, boxcox_lambda, inv_boxcox

from statsforecast import StatsForecast
from statsforecast.models import (
    AutoMFLES,
)

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
group = 'Monthly'  

# for M4
df, *_ = M4.load("data", group)
df['ds'] = df['ds'].astype('int64')
info = M4Info[group]


# for M3
# df, *_ = M3.load("data", group)
# df['ds'] = df.groupby('unique_id').cumcount() + 1
# info = M3Info[group]


h = info.horizon
test = df.groupby("unique_id").tail(h)
train = df.drop(test.index)

In [4]:
horizon = 18
config = {
    'seasonality_weights': [True, False],
    'smoother': [True, False],
    'ma': [12, 6, 3, None],
    'seasonal_period': [None, 12],
    }

model_1 = AutoMFLES(season_length=12, test_size=12, n_windows=2, metric='smape', config=config, alias='model_1')
model_2 = AutoMFLES(season_length=12, test_size=8, n_windows=3, metric='smape', config=config, alias='model_2')
model_3 = AutoMFLES(season_length=12, test_size=6, n_windows=4, metric='smape', config=config, alias='model_3')
model_4 = AutoMFLES(season_length=12, test_size=4, n_windows=6, metric='smape', config=config, alias='model_4')
model_5 = AutoMFLES(season_length=12, test_size=3, n_windows=8, metric='smape', config=config, alias='model_5')
model_6 = AutoMFLES(season_length=12, test_size=2, n_windows=12, metric='smape', config=config, alias='model_6')

sf = StatsForecast(models=[model_1, model_2, model_3, model_4, model_5, model_6], freq=1, n_jobs=-1, verbose=True)

In [ ]:
sf.fit(df = train)

# Generate test predictions
yhat_test = sf.predict(h=horizon)
yhat_test = yhat_test.reset_index()

/opt/conda/envs/llm_env/lib/python3.10/site-packages/statsforecast/core.py:494: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [9]:
dz = yhat_test.merge(test, on=['unique_id', 'ds'])

In [10]:
model_cols = ['model_1', 'model_2', 'model_3', 'model_4', 'model_5', 'model_6']
dz['ensemble'] = dz[model_cols].mean(axis=1)

# SMAPE function
def smape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / denominator) * 100

smape_values = {}
for col in model_cols:
    smape_val = smape(dz['y'], dz[col])
    smape_values[col] = smape_val

smape_ensemble = smape(dz['y'], dz['ensemble'])
smape_values['ensemble'] = smape_ensemble

print("SMAPE values for each model and ensemble:")
for model, value in smape_values.items():
    print(f"SMAPE for {model}: {value:.2f}")

SMAPE values for each model and ensemble:
SMAPE for model_1: 12.92
SMAPE for model_2: 12.85
SMAPE for model_3: 12.72
SMAPE for model_4: 12.69
SMAPE for model_5: 12.70
SMAPE for model_6: 12.74
SMAPE for ensemble: 12.56
